In [ ]:
import torch
import gc
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, models
from torch import nn
from torch import optim
import json
import PIL
from PIL import Image
import io
import cv2
import torchvision.transforms.functional as TF
from torchvision.utils import save_image
from torchvision.transforms import ToPILImage
from matplotlib import pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.utils.data as data
import torch.optim as optim
from torch.optim import lr_scheduler
from tqdm import tqdm
import time
import os
import copy
import pickle
import urllib.request
import requests
from matplotlib.pyplot import imshow
import random
from utils import *
from resnet50_ft_dims_2048 import *

In [ ]:
#Function to display image and predicted class
def imstats(name):
    # read in image and view it
    x = readim(name, forward_normalize) 
    imshow_tensor(x[0], inv_normalize)

    #get predicted class and probability
    prob = lay2(pretrained_model(x.cuda()))
    maxcls = prob.max(1)
    print("Class is {} ({}) with confidence {}%".format(maxcls.indices.item(),class_dict[maxcls.indices.item()],round(maxcls.values.item()*100,4)))

In [ ]:
model_img_size = 224
model_transform = transforms.Compose([transforms.Resize((model_img_size,model_img_size)),
                                         transforms.ToTensor(),
                                         transforms.Normalize(mean=[91.4953, 103.8827, 131.0912],
                                                              std=[1, 1, 1])
                                     ])

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = resnet50_ft("vgg_face_testimages/resnet50_ft_dims_2048.pth")
model.to(device)
model.eval()

In [ ]:
def load_data(img, shape=None):
    short_size = 224.0
    crop_size = shape
    im_shape = np.array(img.size)    # in the format of (width, height, *)
    img = img.convert('RGB')

    ratio = float(short_size) / np.min(im_shape)
    img = img.resize(size=(int(np.ceil(im_shape[0] * ratio)),   # width
                           int(np.ceil(im_shape[1] * ratio))),  # height
                     resample=PIL.Image.BILINEAR)

    x = np.array(img)  # image has been transposed into (height, width)
    newshape = x.shape[:2]
    h_start = (newshape[0] - crop_size[0])//2
    w_start = (newshape[1] - crop_size[1])//2
    x = x[h_start:h_start+crop_size[0], w_start:w_start+crop_size[1]]
    x = x - mean
    return x

def image_encoding(model, images):
    print('==> compute image-level feature encoding.')
    num_faces = len(images)

    im_array = np.array([load_data(img=i, shape=(224, 224, 3)) for i in images])
    im_tensor = torch.Tensor(im_array.transpose(0, 3, 1, 2))
    im_tensor = im_tensor.to(device)
    f  = model(im_tensor)
    classif = f[0]
    feat = f[1].detach().cpu().numpy()[:, :, 0, 0]
    face_feats = feat / np.sqrt(np.sum(feat ** 2, -1, keepdims=True))
    return classif, face_feats

In [ ]:
mean = (131.0912, 103.8827, 91.4953)
batch_size = 1

#image_dir = "vgg_face_testimages/vgg_face2/samples (test set)/tight_crop (used for training)/n000106/"
#input_im = Image.open(image_dir+"0002_01.jpg")

IMG_URL = "https://images.ctfassets.net/p0qf7j048i0q/34947631635447C28C42A07B6D9CE189/74fa2cbbc95f35863902ff37a2238fab/AP578484237061.jpg?w=1000&h=750&fit=fill&fm=webp"
response = requests.get(IMG_URL)
input_im = Image.open(io.BytesIO(response.content))

display(input_im)

out, face_feats = image_encoding(model, [input_im])
print(out)


In [ ]:
pred = out
print(pred.size())

#Use softmax to get predicted probability and view it
lay2 = torch.nn.Softmax(dim=1)
prob = lay2(pred)
print(prob)
maxOcls = prob.max(1)
print(maxOcls)
print(prob[0][1509])